In [1]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file("shakespeare.text", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

2022-12-19 14:54:19.026251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [3]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

2022-12-19 14:54:26.705505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
encoded -= 2 # dropping 0 (pad) and 1 (unknown)
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)
print(n_tokens, dataset_size)

39 1115394


In [5]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [6]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
             metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
history = model.fit(train_set, validation_data=valid_set, epochs=10,
                   callbacks=[model_ckpt])

Epoch 1/10


2022-12-19 14:54:40.282705: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 54467 of 100000
2022-12-19 14:54:49.996957: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


  31247/Unknown - 2122s 67ms/step - loss: 1.4039 - accuracy: 0.5705

INFO:tensorflow:Assets written to: my_shakespeare_model/assets


INFO:tensorflow:Assets written to: my_shakespeare_model/assets


...

2022-12-19 19:38:21.801423: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


31247/31247 [==============================] - ETA: 0s - loss: 1.2566 - accuracy: 0.6041

INFO:tensorflow:Assets written to: my_shakespeare_model/assets


INFO:tensorflow:Assets written to: my_shakespeare_model/assets


31247/31247 [==============================] - 2190s 69ms/step - loss: 1.2566 - accuracy: 0.6041 - val_loss: 1.5690 - val_accuracy: 0.5444


In [8]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer, 
    tf.keras.layers.Lambda(lambda X: X-2),
    model
])

In [9]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 1s 523ms/step


'e'

In [10]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 1, 0, 2, 1, 0, 0, 1]])>

In [11]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0,-1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [12]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [13]:
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature = 0.01))

1/1 [==============================] - 0s 47ms/step
To be or not to be the season.

provost:
i will not see the duke of 


In [14]:
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 46ms/step
To be or not to bertank i think by great mine intict
which no more:



In [15]:
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 47ms/step
To be or not to bef ,mt'&o3g:auy-$
ws-nse?bws3ert--vgerdjw!c-y-ewznq


In [16]:
print(extend_text("Coffee is really good", temperature=0.5, n_chars=100))

1/1 [==============================] - 0s 53ms/step
Coffee is really good as the service.

pompey:
sir, i am better me as in the selfsame of the servant have for the service


In [17]:
print(extend_text("Coffee ", n_chars=100, temperature=0.4))

1/1 [==============================] - 0s 46ms/step
Coffee her is the sun that and the provost in the sound, sir, i have she is the shepherd's great princely d


### Stateful RNN

In [18]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length +1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length +1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000], length)
staeful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16,
                             batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [20]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [21]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
             metrics=["accuracy"])
history = model.fit(stateful_train_set, validation_data=stateful_valid_set,
                   epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])

Epoch 1/10
9999/9999 [==============================] - 226s 22ms/step - loss: 1.8611 - accuracy: 0.4527 - val_loss: 1.7099 - val_accuracy: 0.4914
Epoch 2/10
9999/9999 [==============================] - 206s 21ms/step - loss: 1.5640 - accuracy: 0.5288 - val_loss: 1.6216 - val_accuracy: 0.5136
Epoch 3/10
9999/9999 [==============================] - 218s 22ms/step - loss: 1.4868 - accuracy: 0.5487 - val_loss: 1.5805 - val_accuracy: 0.5265
Epoch 4/10
9999/9999 [==============================] - 207s 21ms/step - loss: 1.4471 - accuracy: 0.5588 - val_loss: 1.5549 - val_accuracy: 0.5312
Epoch 5/10
9999/9999 [==============================] - 210s 21ms/step - loss: 1.4223 - accuracy: 0.5647 - val_loss: 1.5403 - val_accuracy: 0.5346
Epoch 6/10
9999/9999 [==============================] - 225s 22ms/step - loss: 1.4056 - accuracy: 0.5690 - val_loss: 1.5359 - val_accuracy: 0.5352
Epoch 7/10
9999/9999 [==============================] - 217s 22ms/step - loss: 1.3932 - accuracy: 0.5719 - val_loss: 1

In [22]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16,
                             batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=False),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [23]:
model2.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

In [24]:
model2.set_weights(model.get_weights())

In [25]:
def next_char(text, temperature=1):
    y_proba = model2.predict([text])[0,-1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

## Sentiment Analysis

In [26]:
import tensorflow_datasets as tfds

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [27]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8"))
    print("Label:", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development

2022-12-19 20:50:47.398418: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [28]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

In [29]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
             metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 339s 478ms/step - loss: 0.6935 - accuracy: 0.4989 - val_loss: 0.6931 - val_accuracy: 0.5008
Epoch 2/2
704/704 [==============================] - 338s 481ms/step - loss: 0.6917 - accuracy: 0.5139 - val_loss: 0.7414 - val_accuracy: 0.5728


### Masking

In [30]:
inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
token_ids = text_vec_layer(inputs)
mask = tf.math.not_equal(token_ids, 0)
Z = tf.keras.layers.Embedding(vocab_size, embed_size)(token_ids)
Z = tf.keras.layers.GRU(128, dropout=0.2)(Z, mask=mask)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(Z)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

In [31]:
text_vec_layer_ragged = tf.keras.layers.TextVectorization(
    max_tokens = vocab_size, ragged=True)

text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels: reviews))
text_vec_layer_ragged(["Great Movie!", "This is DiCaprio's best role."])

<tf.RaggedTensor [[86, 18], [11, 7, 1, 116, 217]]>

### Reusing Pretrained Embeddings and Language Models

In [32]:
import os 
import tensorflow_hub as hub

os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                  trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
             metrics=["accuracy"])
model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10
704/704 [==============================] - 711s 987ms/step - loss: 0.3139 - accuracy: 0.8668 - val_loss: 0.2699 - val_accuracy: 0.8860
Epoch 2/10
704/704 [==============================] - 693s 985ms/step - loss: 0.0492 - accuracy: 0.9854 - val_loss: 0.3185 - val_accuracy: 0.8956
Epoch 3/10
704/704 [==============================] - 690s 980ms/step - loss: 0.0048 - accuracy: 0.9992 - val_loss: 0.3734 - val_accuracy: 0.8904
Epoch 4/10
704/704 [==============================] - 690s 979ms/step - loss: 9.9242e-04 - accuracy: 0.9999 - val_loss: 0.4423 - val_accuracy: 0.8980
Epoch 5/10
704/704 [==============================] - 690s 980ms/step - loss: 5.1683e-04 - accuracy: 1.0000 - val_loss: 0.4618 - val_accuracy: 0.8960
Epoch 6/10
704/704 [==============================] - 689s 979ms/step - loss: 4.6563e-04 - accuracy: 1.0000 - val_loss: 0.4976 - val_accuracy: 0.8944
Epoch 7/10
704/704 [==============================] - 690s 980ms/step - loss: 3.9346e-04 - accuracy: 1.0000 - va

## An Encoder-Decoder Network for Neural Machine Translation

In [33]:
from pathlib import Path

url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                              extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

In [34]:
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)

In [35]:
for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

Why didn't you let me know? => Por qué no me avisaste?
He assumed a new identity. => Él asumió una nueva identidad.
I don't understand German. => No entiendo el alemán.


In [36]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [37]:
text_vec_layer_en.get_vocabulary()[:10]

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']

In [38]:
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [39]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [40]:
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [41]:
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                   mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                   mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [42]:
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

In [43]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [44]:
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

In [45]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                      outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 1519s 484ms/step - loss: 0.4162 - accuracy: 0.4242 - val_loss: 0.3109 - val_accuracy: 0.5209
Epoch 2/10
3125/3125 [==============================] - 1506s 482ms/step - loss: 0.2663 - accuracy: 0.5694 - val_loss: 0.2394 - val_accuracy: 0.6037
Epoch 3/10
3125/3125 [==============================] - 1504s 481ms/step - loss: 0.2088 - accuracy: 0.6415 - val_loss: 0.2073 - val_accuracy: 0.6485
Epoch 4/10
3125/3125 [==============================] - 1503s 481ms/step - loss: 0.1755 - accuracy: 0.6874 - val_loss: 0.1933 - val_accuracy: 0.6673
Epoch 5/10
3125/3125 [==============================] - 1512s 484ms/step - loss: 0.1520 - accuracy: 0.7213 - val_loss: 0.1871 - val_accuracy: 0.6765
Epoch 6/10
3125/3125 [==============================] - 1505s 482ms/step - loss: 0.1329 - accuracy: 0.7496 - val_loss: 0.1854 - val_accuracy: 0.6815
Epoch 7/10
3125/3125 [==============================] - 1508s 483ms/step - loss: 0.1166 - accuracy: 0.7749

In [46]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])
        X_dec = np.array(["startofseq " + translation])
        y_proba = model.predict((X, X_dec))[0, word_idx]
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

In [47]:
translate("I like soccer")

1/1 [==============================] - 0s 48ms/step


'me gusta el fútbol'

### Bidirectional RNNs

In [48]:
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state=True))

In [50]:
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = (tf.concat(encoder_state[::2], axis=-1), # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)) # long-term (1 & 3)

### Attention Mechanisms

In [51]:
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)
)

In [53]:
attention_layer = tf.keras.layers.Attention()
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(attention_outputs)

#### Positional encodings

In [54]:
max_length = 50
embed_size = 128
pos_embed_layer = tf.keras.layers.Embedding(max_length, embed_size)
batch_max_len_enc = tf.shape(encoder_embeddings)[1]
encoder_in = encoder_embeddings + pos_embed_layer(tf.range(batch_max_len_enc))
batch_max_len_dec = tf.shape(decoder_embeddings)[1]
decoder_in = decoder_embeddings + pos_embed_layer(tf.range(batch_max_len_dec))

In [56]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_length, embed_size, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        assert embed_size % 2 == 0, "embed_size must be even"
        p, i = np.meshgrid(np.arange(max_length), 
                           2 * np.arange(embed_size // 2))
        pos_emb = np.empty((1, max_length, embed_size))
        pos_emb[0, :, ::2] = np.sin(p / 10_000 ** (i / embed_size)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10_000 ** (i / embed_size)).T
        self.pos_encodings = tf.constant(pos_emb.astype(self.dtype))
        self.supports_masking = True
        
    def call(self, inputs):
        batch_max_length = tf.shape(inputs)[1]
        return inputs + self.pos_encodings[:, :batch_max_length]

In [57]:
pos_embed_layer = PositionalEncoding(max_length, embed_size)
encoder_in = pos_embed_layer(encoder_embeddings)
decoder_in = pos_embed_layer(decoder_embeddings)

#### Multi-head attention

In [62]:
# encoder
N = 2
num_heads = 8
dropout_rate = 0.1
n_units = 128
encoder_pad_mask = tf.math.not_equal(encoder_input_ids, 0)[:, tf.newaxis]
Z = encoder_in
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.Dropout(dropout_rate)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

In [63]:
decoder_pad_mask = tf.math.not_equal(decoder_input_ids, 0)[:, tf.newaxis]
causal_mask = tf.linalg.band_part(tf.ones((batch_max_len_dec, batch_max_len_dec), tf.bool), -1, 0)

In [65]:
# decoder
encoder_outputs = Z
Z = decoder_in 
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=causal_mask & decoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=encoder_outputs, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

In [66]:
Y_proba = tf.keras.layers.Dense(vocab_size, activation="softmax")(Z)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                      outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
                 metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 1585s 504ms/step - loss: 0.1420 - accuracy: 0.8339 - val_loss: 0.1344 - val_accuracy: 0.8147
Epoch 2/10
3125/3125 [==============================] - 1723s 551ms/step - loss: 0.0884 - accuracy: 0.8873 - val_loss: 0.0842 - val_accuracy: 0.8906
Epoch 3/10
3125/3125 [==============================] - 1531s 490ms/step - loss: 0.0853 - accuracy: 0.8885 - val_loss: 0.0827 - val_accuracy: 0.8914
Epoch 4/10
3125/3125 [==============================] - 1427s 457ms/step - loss: 0.0824 - accuracy: 0.8899 - val_loss: 0.0811 - val_accuracy: 0.8915
Epoch 5/10
3125/3125 [==============================] - 1593s 510ms/step - loss: 0.0802 - accuracy: 0.8913 - val_loss: 0.0810 - val_accuracy: 0.8914
Epoch 6/10
3125/3125 [==============================] - 1443s 462ms/step - loss: 0.0790 - accuracy: 0.8914 - val_loss: 0.0786 - val_accuracy: 0.8932
Epoch 7/10
3125/3125 [==============================] - 1361s 435ms/step - loss: 0.0774 - accuracy: 0.8927

## Hugging Face's Transformers Library

In [72]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier("The actors were very convincing.")
print(result)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceCla

[{'label': 'POSITIVE', 'score': 0.9998071789741516}]


In [73]:
classifier(["I am from Rochester.", "I am from Iraq."])

[{'label': 'POSITIVE', 'score': 0.9867823123931885},
 {'label': 'NEGATIVE', 'score': 0.9811071157455444}]

In [75]:
model_name = "huggingface/distilbert-base-uncased-finetuned-mnli"
classifier_mnli = pipeline("text-classification", model=model_name)
classifier_mnli("She loves me. [SEP] She loves me not.")

Downloading:   0%|          | 0.00/729 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some layers from the model checkpoint at huggingface/distilbert-base-uncased-finetuned-mnli were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at huggingface/distilbert-base-uncased-finetuned-mnli and are newly initialized: ['dropout_81']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': 'contradiction', 'score': 0.9790192246437073}]

In [78]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

Some layers from the model checkpoint at huggingface/distilbert-base-uncased-finetuned-mnli were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at huggingface/distilbert-base-uncased-finetuned-mnli and are newly initialized: ['dropout_121']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
token_ids = tokenizer(["I like soccer. [SEP] We all love soccer!", 
                       "Joe lived for a very long time. [SEP] Joe is old."],
                     padding=True, return_tensors="tf")
token_ids

{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 2066, 4715, 1012,  102, 2057, 2035, 2293, 4715,  999,
         102,    0,    0,    0],
       [ 101, 3533, 2973, 2005, 1037, 2200, 2146, 2051, 1012,  102, 3533,
        2003, 2214, 1012,  102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [82]:
outputs = model(token_ids)
outputs

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-2.1123827 ,  1.1786778 ,  1.4101031 ],
       [-0.01478416,  1.0962465 , -0.9919941 ]], dtype=float32)>, hidden_states=None, attentions=None)

In [84]:
Y_probas = tf.keras.activations.softmax(outputs.logits)
Y_probas

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.01619699, 0.43523493, 0.548568  ],
       [0.22655973, 0.68817234, 0.08526795]], dtype=float32)>

In [86]:
Y_pred = tf.argmax(Y_probas, axis=1)
Y_pred

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 1])>

In [87]:
sentences = [("Sky is blue", "Sky is red"), ("I love her", "She loves me")]
X_train = tokenizer(sentences, padding=True, return_tensors="tf").data
y_train = tf.constant([0,2])
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=2)

Epoch 1/2
1/1 [==============================] - 20s 20s/step - loss: 0.6666 - accuracy: 0.5000
Epoch 2/2
1/1 [==============================] - 1s 741ms/step - loss: 0.3184 - accuracy: 1.0000
